<a href="https://colab.research.google.com/github/henouji/hnj/blob/master/SouthPark_Chat_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports and Libraries 

In [1]:
import tensorflow as tf 
import numpy as np
from keras.models import Model
from keras.layers import CuDNNLSTM, Input, Dense, LSTM 
from keras.optimizers import RMSprop
import pandas as pd 
from google.colab import files 
import io 

Using TensorFlow backend.


# File upload and data reading

In [2]:
uploaded = files.upload()

Saving sPark.csv to sPark.csv


In [0]:
df = pd.read_csv(io.BytesIO(uploaded['sPark.csv']))

# Data Preparation and Input Data Initialization for Character Level Processing


> Turn the sentences into 3 Numpy arrays, encoder_input_data, decoder_input_data, decoder_target_data:

> * encoder_input_data is a 3D array of shape (num_pairs, max_english_sentence_length, num_english_characters) containing a one-hot vectorization of the English sentences
* decoder_input_data is a 3D array of shape (num_pairs, max_french_sentence_length, num_french_characters) containing a one-hot vectorization of the French sentences
* decoder_target_data is the same as decoder_input_data but offset by one timestep. decoder_target_data[:, t, :] will be the same as decoder_input_data[:, t + 1, :]



In [4]:
df.head()

,Season,Episode,Character,Line
0,10,1,Stan,"You guys, you guys! Chef is going away. \n"
1,10,1,Kyle,Going away? For how long?\n
2,10,1,Stan,Forever.\n
3,10,1,Chef,I'm sorry boys.\n
4,10,1,Stan,"Chef said he's been bored, so he joining a gro..."


In [0]:
dataset_array = df.Line.values 

In [7]:
# Limiting Dataset
dataset_array = dataset_array[:len(dataset_array)//12]
print("Total sentences: ",len(dataset_array))

Total sentences:  5908


In [0]:
en_sample = dataset_array[:len(dataset_array)-1]
de_sample = dataset_array[1 :len(dataset_array)]
for i, sent in enumerate(de_sample):
  de_sample[i] = '\t ' + sent 

In [9]:
en_num_pairs = len(en_sample)
de_num_pairs = len(de_sample)

print("Number of Pairs : [",en_num_pairs,",",de_num_pairs,"]")

Number of Pairs : [ 5907 , 5907 ]


In [10]:
max_en_sentence = max([len(x) for x in en_sample])
max_de_sentence = max([len(x) for x in de_sample])

print("Max Sentence: [",max_en_sentence,",",max_de_sentence,"]")

Max Sentence: [ 937 , 937 ]


In [0]:
en_char = set()
de_char = set()

for i in en_sample:
  for x in i:
    if x not in en_char:
      en_char.add(x)
for i in de_sample:
  for x in i:
    if x not in de_char:
      de_char.add(x)

In [12]:
num_en_char = len(en_char)
num_de_char = len(de_char)

print("Number of Characters: [",num_en_char,":",num_de_char,"]")

Number of Characters: [ 89 : 89 ]


In [13]:
# Array Declaration
en_data_input = np.zeros((en_num_pairs, max_en_sentence, num_en_char), dtype='float32')
de_data_input = np.zeros((de_num_pairs, max_de_sentence, num_de_char), dtype='float32')
decode_target = np.zeros((de_num_pairs, max_de_sentence, num_de_char), dtype='float32')
print("Shapes: [",en_data_input.shape,":",en_data_input.shape,"] Target [",decode_target.shape,"]")

Shapes: [ (5907, 937, 89) : (5907, 937, 89) ] Target [ (5907, 937, 89) ]


In [0]:
# For Vectorizing 
en_char_int = dict()
en_int_char = dict()
de_char_int = dict()
de_int_char = dict()

for i, char in enumerate(en_char):
  en_char_int[char] = i
  en_int_char[i] = char
for i, char in enumerate(de_char):
  de_char_int[char] = i
  de_int_char[i] = char

In [0]:
# One Hot Vectorizing
for i, (en_, de_) in enumerate(zip(en_sample, de_sample)):
  for char, char_en in enumerate(en_):
    en_data_input[i, char, en_char_int[char_en]] = 1 
  for char, char_de in enumerate(de_):
    de_data_input[i, char, de_char_int[char_de]] = 1
    if char > 0:
      decode_target[i, char-1, de_char_int[char_de]] = 1 

# Model Building

* Input : encoded data, decoded data
* Output : decode target

In [0]:
# Training Variable Declaration 
batch_size = 128  
epochs = 100 
latent_dim = 256
num_samples = 4000 

In [0]:
en_inputs = Input(shape=(None, num_en_char))

encoder = CuDNNLSTM(latent_dim, return_state=True)

encoder_outputs, state_h, state_c = encoder(en_inputs)

encoder_states = [state_h, state_c]

In [0]:
de_inputs = Input(shape=(None, num_de_char))

decoder_lstm = CuDNNLSTM(latent_dim, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(de_inputs, initial_state=encoder_states)

decoder_dense = Dense(num_de_char, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)

In [0]:
# Combine Encoder and Decoder Model
model = Model([en_inputs, de_inputs], decoder_outputs)

In [29]:
# Training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([en_data_input, de_data_input], decode_target, batch_size=batch_size,
         epochs=epochs, validation_split=0.2)

Train on 4725 samples, validate on 1182 samples
Epoch 1/100
4725/4725 [==============================] - 15s 3ms/step - loss: 0.2222 - val_loss: 0.2079
Epoch 2/100
4725/4725 [==============================] - 14s 3ms/step - loss: 0.2076 - val_loss: 0.2021
Epoch 3/100
4725/4725 [==============================] - 14s 3ms/step - loss: 0.2005 - val_loss: 0.1925
Epoch 4/100
4725/4725 [==============================] - 14s 3ms/step - loss: 0.1895 - val_loss: 0.1832
Epoch 5/100
4725/4725 [==============================] - 14s 3ms/step - loss: 0.1800 - val_loss: 0.1770
Epoch 6/100
4725/4725 [==============================] - 14s 3ms/step - loss: 0.1722 - val_loss: 0.1668
Epoch 7/100
4725/4725 [==============================] - 14s 3ms/step - loss: 0.1663 - val_loss: 0.1636
Epoch 8/100
4725/4725 [==============================] - 14s 3ms/step - loss: 0.1616 - val_loss: 0.1589
Epoch 9/100
4725/4725 [==============================] - 14s 3ms/step - loss: 0.1578 - val_loss: 0.1548
Epoch 10/100
472

In [30]:
# Save model
model.save('s2s.h5')

/usr/local/lib/python3.6/dist-packages/keras/engine/network.py:877: UserWarning: Layer cu_dnnlstm_4 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'cu_dnnlstm_3/strided_slice_16:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'cu_dnnlstm_3/strided_slice_17:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


In [0]:
# Sampling models 
encoder_models = Model(en_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    de_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([de_inputs] + decoder_states_inputs,
                     [decoder_outputs] + decoder_states)

# Model Prediction Testing 

In [0]:
def decode_sequence(input_seq):
  states_value = encoder_models.predict(input_seq)
  
  target_seq = np.zeros((1, 1, num_de_char))
  target_seq[0, 0, de_char_int['\t']] = 1
  
  stop_condition = False 
  decoded_sentence = ''
  
  while not stop_condition:
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
    
    sampled_char_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = de_int_char[sampled_char_index]
    decoded_sentence += sampled_char
    
    if(sampled_char == '\n' or len(decoded_sentence) > max_de_sentence):
      stop_condition = True
      
    target_seq = np.zeros((1, 1, num_de_char))
    target_seq[0, 0, sampled_char_index] = 1
    
    states_value = [h, c]
    
  return decoded_sentence

In [0]:
def sent_seq(sentence):
  input_seq = np.zeros((1, max_en_sentence, num_en_char))
  for i, letter in enumerate(sentence):
    input_seq[0, i, en_char_int[letter]]
  return input_seq

In [0]:
for seq_index in range(10):
  input_seq = en_data_input[seq_index : seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)
  
  print('-')
  print('Input:', en_sample[seq_index])
  print('Decode: ', decoded_sentence)

In [42]:
query = input(": ")
input_seq = sent_seq(query)
print(">", decode_sequence(input_seq))

: He's going to join the army
>  What? What's the matter? 

